In [1]:
import pandas as pd
import numpy as np
import regex as re
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,Bidirectional, GlobalMaxPooling1D,Dropout,Flatten,Conv1D,MaxPooling1D
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
data = pd.read_csv('version-4.csv',index_col=0)
data.reset_index(drop=True,inplace = True)

for i,j in enumerate(data['labels']):
  try:
    j = int(j)
  except ValueError as e:
    print(f'error on {i} line')
data.drop(labels=[2478],inplace= True)
data['labels'].astype(int);
data.dropna(inplace = True)

error on 2478 line


In [3]:
cleaned_text = []
for text in data['texts']:
  text = " ".join(word for word in text.split() if not word.isdigit())
  cleaned_text.append(text)
data['cleaned_text'] = cleaned_text

In [4]:
vocab =  {}
for text in data['cleaned_text']:
  sen = text.split()
  for word in sen:
    try:
      vocab[word] += 1
    except KeyError:
      vocab[word] = 1
vocab = dict(sorted(vocab.items(), key=lambda item: item[1]))

In [36]:
rare_words = []
for key,value in vocab.items():
  if value<=10:
    rare_words.append(key)

In [37]:
stopwords_en = set(stopwords.words('english'))
cleaner_text = []
for text in data['cleaned_text']:
  text = " ".join([word for word in text.split() if len(word)>2 and word not in rare_words])
  cleaner_text.append(text)
data['final_text'] = cleaner_text

In [38]:
vocab =  {}
for text in data['final_text']:
  sen = text.split()
  for word in sen:
    try:
      vocab[word] += 1
    except KeyError:
      vocab[word] = 1
vocab = dict(sorted(vocab.items(), key=lambda item: item[1]))

In [39]:
vocab_list = list(vocab.items())
vocab_size = len(vocab)

In [46]:
x = data['final_text'].values
y = data['labels'].values
X_train,X_test,y_train, y_test = train_test_split(x,y, test_size = 0.2, shuffle = True)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [45]:
embeddings_index = dict()
f = open('glove.twitter.27B.200d.txt')
for line in f:
  values = line.split()
  word = values[0]
  try:
    coefs = np.asarray(values[1:], dtype='float32')
  except ValueError:
    pass
  embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 26227 word vectors.


In [47]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(X_train,maxlen=200)
X_test = sequence.pad_sequences(X_test,maxlen=200)

In [48]:
tokens = len(tokenizer.word_index) + 2
embedding_matrix = np.zeros((tokens, 200))
count = 0
unknown = []
for word, i in tokenizer.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    try:
      embedding_matrix[i] = embedding_vector
    except ValueError:
      unknown.append(word)
      count += 1
  else:
    unknown.append(word)
    count += 1

In [49]:
print(1-(count/vocab_size))

0.46176411760784053


In [50]:
model = Sequential()
model.add(Embedding(tokens,200,weights = [embedding_matrix],input_length = embedding_matrix.shape[1]))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 200)          2579800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 2,700,301
Trainable params: 2,700,301
Non-trainable params: 0
_________________________________________________________________
None


In [51]:
model.fit(X_train,y_train,epochs=30)
loss,accuracy = model.evaluate(X_train,y_train)
print(f'acc: {accuracy}')

Epoch 1/30
323/323 [==============================] - 14s 40ms/step - loss: 0.4307 - accuracy: 0.7924
Epoch 2/30
323/323 [==============================] - 13s 40ms/step - loss: 0.2452 - accuracy: 0.9002
Epoch 3/30
323/323 [==============================] - 13s 40ms/step - loss: 0.1789 - accuracy: 0.9344
Epoch 4/30
323/323 [==============================] - 13s 40ms/step - loss: 0.1249 - accuracy: 0.9576
Epoch 5/30
323/323 [==============================] - 13s 41ms/step - loss: 0.1050 - accuracy: 0.9625
Epoch 6/30
323/323 [==============================] - 13s 41ms/step - loss: 0.1396 - accuracy: 0.9511
Epoch 7/30
323/323 [==============================] - 13s 39ms/step - loss: 0.0982 - accuracy: 0.9632
Epoch 8/30
323/323 [==============================] - 13s 39ms/step - loss: 0.0810 - accuracy: 0.9711
Epoch 9/30
323/323 [==============================] - 12s 38ms/step - loss: 0.0687 - accuracy: 0.9764
Epoch 10/30
323/323 [==============================] - 13s 39ms/step - loss: 0.067

In [52]:
predictions = model.predict(X_test)
predictions = np.round(predictions)

In [53]:
from sklearn.metrics import accuracy_score, confusion_matrix
score = accuracy_score(y_test,predictions)
cm = confusion_matrix(y_test,predictions)
print("score: {} cm: {}".format(score,cm))

score: 0.8750483558994198 cm: [[1191  179]
 [ 144 1071]]
